# Title

* **Amazon Sales Report SQL Analysis**

# Introduction

* **Objective:** This analysis aims to explore the Amazon Sales Report dataset to derive insights related to sales performance, order fulfillment, and customer distribution across regions. The SQL queries used in this analysis help identify key metrics such as total sales, order status distribution, and regional sales breakdown.

# Dataset Overview

**Dataset Description:** The Amazon Sales Report dataset provides a comprehensive view of transactions made on the Amazon platform. It captures essential details related to orders, including fulfillment methods, product categories, shipping details, and financial data. This dataset is valuable for analyzing sales performance, understanding customer behavior, and assessing the efficiency of fulfillment processes.

**Columns:**
1.  **Order_ID**: A unique identifier for each order, used to track and differentiate transactions.
1.  **Date**: The date when the order was placed, formatted as YY-MM-DD. This allows for time-based analysis, such as trends in sales over time or the impact of specific dates on sales volumes.
1.  **Status**:  Indicates the current state of the order.Common statuses include 
     *     **Shipped:** The order has been dispatched.
     *     **Cancelled:** The order was cancelled and not processed further.
     *     **Delivered:** The order has been delivered to the customer.
1.  **Fulfilment**: Specifies whether the order was fulfilled by Amazon or by a third-party Merchant. This column helps in assessing the efficiency and reliability of different fulfillment methods. 
1.  **Sales_Channel**: The platform or channel through which the sale was made (e.g., Amazon.in). This information is useful for analyzing the performance of different sales channels  
1.  **ship_service_level**: The level of shipping service selected for the order, such as **Standard** or **Expedited**. This can be analyzed to understand customer preferences for shipping speed.
1.  **Category**: The product category to which the ordered item belongs, such as **T-shirt**, **Shirt**, or **Blazzer**. This helps in identifying which product categories are most popular among customers.
1.  **Size**: The size of the product, which is particularly relevant for apparel items. Understanding size distribution can inform inventory decisions and customer preferences.
1.  **Courier Status**: The current status of the shipment, such as **On the Way** or **Shipped**. This column provides insight into the shipping process and potential delays.
1.  **Qty**: The quantity of items ordered in a particular transaction. This is useful for calculating total units sold and understanding order patterns.
1.  **Amount**: The total monetary value of the order. This column is critical for revenue analysis and calculating metrics like total sales and average order value.
1.  **ship_city**: The city to which the order was shipped. Analyzing sales by city can reveal geographic trends and hotspots for customer activity.
1.  **ship_state**: The state to which the order was shipped. This column is key for regional analysis, helping to identify which states contribute most to sales and where marketing efforts could be focused
1.  **ship_postal-code**: The postal code of the shipping address, which can be used for more granular geographic analysis.



**Potential Uses:**

* **Sales Performance Analysis**: Measure total sales, average order values, and trends over time.
* **Customer Behavior Analysis**: Understand customer preferences based on product categories, sizes, and shipping methods.
* **Operational Efficiency**: Evaluate the performance of different fulfillment methods and shipping services.
* **Geographic Insights**: Identify high-performing regions and potential markets for expansion.

This dataset serves as a robust foundation for various types of analysis, providing insights that can help improve sales strategies, customer satisfaction, and operational efficiency on the Amazon platform.

# Data cleaning


* **Handling duplicates**: As Order_id column is unique indetifier for each order so we will be checking for this column. I expect there should be one order_id per order.

In [ ]:
SELECT Order_ID,
        COUNT(*) AS count
FROM Amazon_sales
GROUP BY Order_ID
HAVING  count(*) >1; 

**Output:** 6988 duplicate records found.


Treating the duplicate values by deleting them.

In [ ]:
WITH CTE AS (
    SELECT 
        Order_ID, 
        ROW_NUMBER() OVER (PARTITION BY Order_ID ORDER BY (SELECT NULL)) AS rn
    FROM 
        Amazon_sales
)
DELETE 
FROM CTE 
WHERE rn > 1;

* **Handling missing data**  
     Firstly, Let's check the NULL values in key columns such as ship state and Amount column which we will be using for further analysis

In [ ]:
SELECT 
    COUNT(*) AS Total_Records, 
    SUM(CASE WHEN ship_state IS NULL THEN 1 ELSE 0 END) AS Null_Count
FROM Amazon_Sales;


* NULL values handling: The **ship_state** and **Amount** column had null values, which were handled by replacing them with suitable data. The SQL command used for this as follows:

In [ ]:
# Replacing Null value in 'ship_state' with 'Unknown' 
UPDATE Amazon_Sales
SET ship_state = 'Unknown'
WHERE ship_state IS NULL;

# Replacing Null value in 'Amount' with '0'
UPDATE Amazon_Sales
SET Amount = 0
WHERE Amount IS NULL;

* **Derived column**

* Total_Amount = The "Total_Amount" column is a calculated field that represents the total monetary value of each order. It is derived by multiplying the quantity of items ordered (Qty) by the price per unit. This column is crucial for accurately reflecting the revenue generated from each transaction.The SQL query for this as follows:

In [ ]:
#Creating the Column 'Total_Amount' and 'Float' as data type
ALTER TABLE amazon_sales
ADD Total_Amount FLOAT;

#updating the records in 'Total_Amount' column
UPDATE Amazon_sales SET Total_amount= Amount*Qty;




* Order_status: By adding a new column "Order_Status" and populating it based on the current Status and courier_status fields. The Order_Status is categorized into specific values: 'Successful_Delivery', 'Delivery_issues', 'Order_Processing', 'Shipping_in_progress', and 'Order_Cancelled', depending on the conditions in the existing data, thereby enabling easier tracking and analysis of order progress and issues.The Query for this as follows:

In [ ]:
#Creating a column 'Order_status' and adding data type
ALTER TABLE Amazon_sales
ADD Order_Status varchar(max);

#Updating the 'Order_status' column with records
UPDATE Amazon_sales SET Order_Status =(CASE when Status IN ('Shipped - Delivered to Buyer') THEN 'Successful_Delivery'
              WHEN Status IN ('Shipped - Returned to Seller','Shipped - Returning to Seller','Shipped - Damaged','Shipped - Rejected by Buyer','Shipped - Lost in Transit')  THEN 'Delivery_issues'
              WHEN Status IN ('Pending','Pending - Waiting for Pick Up') OR courier_status ='Unshipped' THEN 'Order_Processing'
              WHEN Status IN ('Shipping','Shipped','Shipped - Out for Delivery','Shipped - Picked Up') OR courier_status IN ('Shipped','On the Way') THEN 'Shipping_in_progress'
              ELSE 'Order_Cancelled' END ) FROM Amazon_sales;

# Analyse the data

 **1. Total Sales Amount:** This represents the total revenue generated from all orders in the dataset. It is a sum of the Total_Amount column, which reflects the total monetary value of each transaction. 

In [ ]:
SELECT ROUND(SUM(Amount),2) AS Total_Sales_Amount
FROM Amazon_Sales;

**Output:** 7,33,23,468.87 rupees

**2. Number of Orders:** This counts the total number of unique orders placed, as identified by the Order ID column.

In [ ]:
SELECT COUNT(DISTINCT Order_ID) AS Number_of_Orders
FROM Amazon_Sales;

**Output:** 1,20,229 units 

**3. Average Order Value (AOV):** The AOV is calculated by dividing the total sales amount by the number of orders. It provides insight into the average amount spent per order.

In [ ]:
SELECT ROUND(SUM(Amount) / COUNT(DISTINCT Order_ID),2) AS Average_Order_Value
FROM Amazon_sales;

**Output:** 609.87 rupees

**4. Order Fulfilment Rate:** This shows the distribution of orders between those fulfilled by Amazon and those fulfilled by third-party merchants, as captured in the Fulfilment column.

In [ ]:
SELECT Fulfilment, COUNT(*) AS Order_Count,
       ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) over()),2) AS Fulfilment_Rate
FROM Amazon_sales
GROUP BY Fulfilment;

**Output:**

| Fulfilment | Order_Count | Fulfilment_Rate |
|------------|-----------------------|-------|
| Amazon     | 83906                 | 69.79 |
| Merchant   | 36323                 | 30.21 |



**5. Order Status Breakdown:** This analysis provides a breakdown of orders by their status, such as Shipped, Cancelled, or Delivered.

In [ ]:
SELECT Order_Status, COUNT(*) AS Order_Count,
       ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER()),2) AS Status_Percentage
FROM Amazon_Sales
GROUP BY order_Status;

**Output:**

| Order_Status         | Order_Count | Status_Percentage |
|----------------------|-------------|-------------------|
| Successful_Delivery  | 26521       | 22.06             |
| Shipping_in_progress | 79952       | 66.50             |
| Order_Cancelled      | 5571        | 4.63              |
| Order_Processing     |6189         | 5.15              |
| Delivery_issues      |1996         | 1.66              |

**6. Sales by Region (State):** This analysis shows the total sales amount and order count for each state, helping to identify regional sales performance.

In [ ]:
SELECT ship_state, ROUND(SUM(Amount),2) AS Total_Sales_Amount,
       COUNT(*) AS Number_of_Orders,
       ROUND((SUM(Amount) * 100.0 / SUM(SUM(Amount)) OVER()),2) AS Sales_Percentage_by_State
FROM Amazon_Sales
GROUP BY ship_state
ORDER BY Sales_Percentage_by_State DESC;

**Output:**

| ship_state     | Total_Sales_Amount | Number_of_Orders | Sales_Percentage_by_State |
|------------    |--------------------|------------------|---------------------------|
| MAHARASHTRA    | 12440118.52        | 20756            | 16.97                     |
| KARNATAKA      | 	9817768.1         |16162	         | 13.39                     |
| UTTAR PRADESH  | 	6437670.88        | 	10057        |	8.78                     | 
| TELANGANA      | 	6331438.99        | 	10394        |	8.63                     |
| TAMIL NADU     | 	5965184.37        | 	10507        | 	8.14                     | 
| DELHI          | 	4069730.06        | 	6520         | 	5.55                     | 
| KERALA         | 	3564014.15        | 	6097         | 	4.86                     | 
| WEST BENGAL    | 	3318224.53        | 	5645         | 	4.53                     | 
| ANDHRA PRADESH | 	2945864.69        | 	4970         | 	4.02                     | 
| HARYANA        | 	2702758.71        | 	4134         | 	3.69                     | 
| GUJARAT        | 	2525196.52        | 	4161         | 	3.44                     | 
| RAJASTHAN      | 	1623524.21        | 	2509         | 	2.21                     |  
| MADHYA PRADESH | 	1491440.85        | 	2375         | 	2.03                     | 
| BIHAR          | 	1348952.26        | 	2014         | 	1.84                     | 
| ODISHA         | 	1313765.86        | 	2023         | 	1.79                     | 
| PUNJAB         | 	1139155.78        | 	1808         | 	1.55                    | 
| ASSAM	         | 983227.48          | 	1601         | 	1.34                      | 
| UTTARAKHAND    | 	924741.74         | 	1478         | 	1.26                      | 
| JHARKHAND      | 	873999.95         | 	1381         | 	1.19                      | 
| GOA            | 	606016.85         | 	1079         | 	0.83                      | 
| CHHATTISGARH      | 	546413.19 | 	870 | 	0.75 |
| HIMACHAL PRADESH  | 	477558.22 | 	750 | 	0.65 |
| JAMMU & KASHMIR   | 	434705.93 | 	669 | 	0.59 |
| MANIPUR           | 	194770.99 | 	285 | 	0.27 |
| CHANDIGARH        | 	198959.67 | 	315 | 	0.27 |
| PONDICHERRY       | 	167583.29 | 	307 | 	0.23 |
| ANDAMAN & NICOBAR | 	148225.81 | 	241 | 	0.2  |
| SIKKIM            | 	135579.66 | 	197 | 	0.18 | 
| NAGALAND          | 	127810.67 | 	170 | 	0.17 | 
| MEGHALAYA         | 	112533.81 | 	194 | 	0.15 | 
| TRIPURA           | 	87549.83  | 	143 | 	0.12 | 
| ARUNACHAL PRADESH | 	88815     | 	134 | 	0.12 | 
| NEW DELHI         | 	45470.95  | 	76  | 	0.06 | 
| MIZORAM           | 	40686.71  | 	72  | 	0.06 | 
| LADAKH            | 	37738.43  | 	42  | 	0.05 | 
| DADRA AND NAGAR   | 	37838.92  | 	60  | 	0.05 | 
| UNKNOWN           | 	15258     | 	28  | 	0.02 | 
| LAKSHADWEEP       | 	3175.29   | 	4   | 	0    | 
| APO               | 	0         | 	1   | 	0    | 

**7. Cancelled Orders Rate:** This analysis measures the percentage of orders that were cancelled, using the Status column.

In [ ]:
SELECT COUNT(*) AS Cancelled_Orders_Count,
       ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Amazon_Sales)),2) AS Cancelled_Order_Rate
FROM Amazon_Sales
WHERE Order_Status = 'Order_Cancelled';

**Output:**

* Cancelled_Orders_Count = 5571
* Cancelled_Order_Rate = 4.63 


**8. Quantity Sold:** This analysis sums the Qty column to provide the total number of units sold.

In [ ]:
SELECT SUM(Qty) AS Total_Quantity_Sold
FROM Amazon_Sales;

**Output:** 1,08,637 units

**9. Category Sales Distribution:** This analysis explores the distribution of sales across different product categories, as specified in the Category column.

In [ ]:
SELECT Category, ROUND(SUM(Amount),2) AS Total_Sales_Amount,
       COUNT(*) AS Number_of_Orders,
       ROUND((SUM(Amount) * 100.0 / SUM(SUM(Amount)) OVER()),2) AS Category_Sales_Percentage
FROM Amazon_Sales
GROUP BY Category
ORDER by Category_Sales_Percentage DESC;

**Output:**

| Category |	Total_Sales_Amount |	Number_of_Orders |	Category_Sales_Percentage |
|----------|-----------------------|---------------------|----------------------------|
|T-shirt	|36684084	|47145|	50.03|
|Shirt|	19574797.54|	45856|	26.7 |
|Blazzer|	10622129.15|	14703|	14.49|
|Trousers|	5024752.81|	10008|	6.85|
|Perfume|	730727.61|	1079|	1|
|Wallet|	441243.52|	890|	0.6|
|Socks|	137551.48|	404|	0.19|
|Shoes|	107877.76|	143|	0.15|
|Watch|	305|	1|	0|

**10. Shipping Method Usage:** This analysis examines the preference for different shipping methods, using the ship-service-level column.

In [ ]:
SELECT ship_service_level, COUNT(*) AS Order_Count,
       ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER()),2) AS Shipping_Method_Percentage
FROM Amazon_Sales
GROUP BY ship_service_level;

**Output:**

|ship_service_level|	Order_Count|	Shipping_Method_Percentage|
|------------------|---------------|------------------------------|
|Expedited|	82827|	68.89|
|Standard|	37402|	31.10|

**11. Sales Trends Over Time:** This analysis tracks sales performance over time by aggregating sales by month or day, using the Date column.

In [ ]:
SELECT DATENAME(MONTH,order_Date) AS Month,
       ROUND(SUM(total_amount),2) AS Total_sales_Amount,
       COUNT(*) AS No_of_Orders,
        ROUND((SUM(total_amount) *100/SUM(SUM(total_amount)) over()),2)AS sales_percent_by_month
FROM Amazon_sales
GROUP BY DATENAME(MONTH,order_Date),
         MONTH(order_date)
ORDER BY MONTH(order_date); 

**Output:**

|Month |	Total_sales_Amount |	No_of_Orders |	sales_percent_by_month |
|------|-----------------------|-----------------|--------------------|
|March	|92549	|158	|0.13|
|April|	25886087|	45709|	36.53|
|May	|23619231	|39221	|33.33|
|June|	21270794|	35141|	30.01|

# Conclusion

The comprehensive analysis of the Amazon Sales Report dataset reveals several key insights that can inform strategic decisions and operational improvements for the business. By leveraging the KPIs and further analysis, the following conclusions can be drawn:

**1. Overall Sales Performance:**
*     The **Total Sales Amount of 7,33,23,468.87 rupees** indicates robust revenue generation, reflecting a healthy volume of transactions. Monitoring this metric over time can help in identifying periods of peak sales activity and understanding the factors driving these results.

* The **Average Order Value (AOV) of 609.87 rupees** provides a clear picture of customer spending behavior. A consistent or increasing AOV suggests effective pricing strategies and customer willingness to purchase higher-value items or bundles.

**2. Customer and Order Behavior:**
*     A total of **1,20,229 units** unique orders were processed, as reflected in the **Number of Orders** metric. This volume shows consistent customer engagement and order flow.

* The **Order Status Breakdown** revealed that **66.50%** of orders were **successfully shipped**, **4.63%** were **cancelled**, and **22.06%** were **delivered**. The relatively low cancellation rate suggests strong inventory management and customer satisfaction.

**3. Regional Sales Insights:**
*     The **Sales by Region (State)** analysis identified that the state of **Maharashtra** contributed **124,40,118.52rupees** in sales, representing **16.97%** of total sales. This makes it the top-performing region, indicating a potential focus area for targeted marketing and logistics efforts.
    
*     Other high-performing states, such as **Karnataka** and **Uttar Pradesh**, also showed significant contributions, with sales amounting to **98,17,768.1rupees** and **64,37,670.88rupees**, respectively. These insights are vital for regional strategy development.

**4. Fulfillment and Shipping:**
*     The **Order Fulfillment Rate** analysis showed that **69.79%** of orders were fulfilled by **Amazon**, while **30.21%** were fulfilled by **third-party merchants**. This split indicates a balanced approach to order fulfillment, with Amazon maintaining a strong role in ensuring timely deliveries.

**5. Sales Trends Over Time:**
*     The **Sales Trends Over Time** analysis showed peak sales periods in **April**, with total monthly sales reaching **258,86,087rupees**. Identifying these trends helps in preparing for high-demand periods, ensuring adequate inventory and staffing+

**6. Operational Efficiency:**
*     The **Cancelled Orders Rate** was found to be **4.63%**, which is relatively low and indicates effective order processing and customer satisfaction. Reducing this rate further could lead to even higher customer retention and revenue stability.


# Final Recommendations

Given these insights, the following strategic actions are recommended:

* **Focus on High-Performing Regions:** Enhance marketing and logistics operations in top-performing states like **Maharashtra,** **Karnataka**, and **Uttar Pradesh** to drive further growth in these areas.

* **Optimize Fulfillment Processes:** Continue to improve fulfillment operations to maintain and potentially increase the proportion of Amazon-fulfilled orders, ensuring timely and reliable deliveries.

* **Capitalize on Seasonal Trends:** Use the identified sales trends to anticipate demand spikes during peak periods, allowing for better inventory management and staffing adjustments.

* **Leverage Shipping Preferences:** Tailor shipping options based on customer preferences, offering promotions for faster shipping services where there is clear demand.

By focusing on these areas, the company can maintain its competitive edge, improve customer satisfaction, and drive sustained growth. Regular monitoring of these periodic deeper analysis will be essential to adapting to market changes and seizing new opportunities.

**Thankyou for reading my first SQL project , I appreciate it!**